# Parse SOOC Reports

This notebooks takes the historical State of our City reports and parses them into flat, chunked formats ready for vector search. It mainly used DBSQL to accomplish this.

## Read
This first cell loads all the PDF reports into a `Bytes` table that can be read by the parsing engine

In [0]:
CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.bytes') AS 
  SELECT
    path,
    modificationTime,
    length,
    _metadata,
    content
  FROM READ_FILES(
    :doc_path, 
    format => 'binaryFile', 
    recursiveFileLookup => true
    )

## Parse
This next cell uses Databricks' AI Parse function to parse the documents.

In [0]:
CREATE OR REPLACE TABLE IDENTIFIER(:catalog || '.' || :schema || '.parsed') AS (
  SELECT
    path,
    AI_PARSE_DOCUMENT(
      content,
      map(
        'imageOutputPath', CAST(:image_path AS STRING),
        'descriptionElementTypes', '*'
      )
    ) AS parsed
  FROM (
    SELECT path, content 
    FROM IDENTIFIER(:catalog || '.' || :schema || '.bytes')
  )
)

In [0]:
SELECT * 
FROM IDENTIFIER(:catalog || '.' || :schema || '.parsed')
LIMIT 3

## FLATTEN
We now have a convoluted json structure with all of our document information, but need to flatten it, with bounding boxes and files for the figures so researchers can reference them. 